In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def convert_time(df, column):
    col_lst = [datetime.fromtimestamp(i) for i in df[column]]
    df[column] = pd.Series(col_lst)

In [3]:
file = 'data/Kickstarter_2019-10-17T03_20_19_421Z.json'
df0 = pd.read_json(file, lines=True)

In [4]:
df = pd.DataFrame(list(df0.data))

In [5]:
#Inspect all columns of dataframe
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'country_displayable_name', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [6]:
#Inspect dataframe shape
df.shape

(205227, 38)

In [7]:
#Check content within category column
df.category[0]

{'id': 270,
 'name': 'Gaming Hardware',
 'slug': 'games/gaming hardware',
 'position': 1,
 'parent_id': 12,
 'color': 51627,
 'urls': {'web': {'discover': 'http://www.kickstarter.com/discover/categories/games/gaming%20hardware'}}}

It seems within category column, 'name' listed the sub category of the project, while 'slug' before the '/' listed the main category. Retract those two information as main_cat and sub_cat for the dataframe and discard rest of category.

In [8]:
main_cat_lst = [i['slug'].split('/')[0] for i in df['category']]
df['main_cat'] = main_cat_lst
df.main_cat.head()

0    games
1    music
2    music
3      art
4    music
Name: main_cat, dtype: object

In [9]:
sub_cat_lst = [i['name'] for i in df.category]
df['sub_cat'] = sub_cat_lst
df.sub_cat.head()

0    Gaming Hardware
1              Metal
2               Jazz
3        Mixed Media
4               Jazz
Name: sub_cat, dtype: object

In [10]:
# Use the convert_time function defined above to convert time series into datatime format
convert_time(df, 'created_at')
convert_time(df, 'launched_at')
convert_time(df, 'deadline')
convert_time(df, 'state_changed_at')

In [11]:
# Check if datatime conversion is successful
df.created_at.head()

0   2017-04-11 01:28:29
1   2013-05-23 12:23:41
2   2013-08-16 08:13:59
3   2018-07-10 13:38:49
4   2013-04-28 20:31:00
Name: created_at, dtype: datetime64[ns]

In [12]:
# Inspect all money related columns
df.loc[:5, ['name', 'main_cat', 'pledged', 'goal', 'converted_pledged_amount', 'usd_pledged', 'currency', 'current_currency', 'usd_type', 'fx_rate', 'static_usd_rate']]

,name,main_cat,pledged,goal,converted_pledged_amount,usd_pledged,currency,current_currency,usd_type,fx_rate,static_usd_rate
0,Playable Certification (Canceled),games,24989.0,375000.0,2963,2963.98652185,SEK,USD,domestic,0.101884,0.118612
1,Deadiron's Next Album!,music,3485.0,1000.0,3485,3485.0,USD,USD,domestic,1.000000,1.000000
2,Lindsey Holland Rick Holland Christmas Project...,music,3575.0,3500.0,3575,3575.0,USD,USD,international,1.000000,1.000000
3,The Belly of the Whale is the Belly of You,art,5523.0,5500.0,5523,5523.0,USD,USD,domestic,1.000000,1.000000
4,"Quartet Style... Recording my jazz vocal CD ""B...",music,100.0,9000.0,100,100.0,USD,USD,international,1.000000,1.000000
5,GETTING BY Language: Spanish 1,publishing,50.0,15000.0,50,50.0,USD,USD,domestic,1.000000,1.000000


In [13]:
# Inspect money related columns for current currency is not 'USD'
df[df.current_currency != 'USD'].loc[:, ['name', 'main_cat', 'pledged', 'goal', 'converted_pledged_amount', 'usd_pledged', 'currency', 'current_currency', 'usd_type', 'fx_rate', 'static_usd_rate']].head()

,name,main_cat,pledged,goal,converted_pledged_amount,usd_pledged,currency,current_currency,usd_type,fx_rate,static_usd_rate
1867,Project Space Boost,games,1.0,500.0,1,1.0,USD,CAD,None,1.320395,1.000000
1995,Surprise Senpai - An Adult Visual Novel/RPG/Da...,games,1611.0,2500.0,2356,1772.4819681,EUR,CAD,None,1.462672,1.100237
2162,The Beast Within,games,1.0,1506.0,1,1.0,USD,CAD,None,1.320395,1.000000
2373,ilumee: 4K LED Projector with Ultimate Short T...,technology,35683.0,50000.0,32212,35683.0,USD,EUR,None,0.902728,1.000000
2688,Amazing Original Vector Logo Design and Branding,design,791.0,100.0,714,791.0,USD,EUR,None,0.902728,1.000000


For currency related columns, there are ‘currency’, ‘currency_symbol’, ‘current_currency’, ‘fx_rate’, ‘static_usd_rate’, ‘pledged’, ‘goal’, ‘usd_pledged’. After checking those columns, I get the information that ‘pledged’ and ‘goal’ are both using ‘currency’ as units, while ‘static_usd_rate’ is the exchange rate between ‘currency’ and US dollar. If the currency column is USD, this rate will be 1.0, and the ‘usd_pledged’ column is simply ‘pledged’ times ‘static_usd_rate’. While ‘current_currency’ for most of the time is USD, but sometimes is other currency, and for some cases ‘currency’ is USD while ‘current currency’ is some other currency. I didn’t figure out what information ‘fx_rate’ and ‘current_currency’ is providing, so I put them to the drop list, still saved in the full version of the cleaned dataset, but are excluded in the useful version. For this project, I think ‘pledged’, ‘goal’, ‘currency’, ‘usd_pledged’ and ‘static_usd_rate’ can provide enough information for my analysis.


In [14]:
%%time
# Retract location name and state from location dictionary
loc_name_lst = [None]*len(df)
state_lst = [None]*len(df)
for index, row in df.iterrows():
    if type(row['location']) == dict:
        loc_name_lst[index] = row['location']['name']
        state_lst[index] = row['location']['state']

Wall time: 20.5 s


In [15]:
df['loc_name'] = loc_name_lst
df['loc_state'] = state_lst

In [16]:
# Check location retract result
df.loc[:10, ['loc_name', 'state', 'country', 'country_displayable_name']]

,loc_name,state,country,country_displayable_name
0,Uppsala,canceled,SE,Sweden
1,Cleveland Heights,successful,US,the United States
2,Buffalo,successful,US,the United States
3,Manhattan,successful,US,the United States
4,Hollywood,failed,US,the United States
5,Los Angeles,failed,US,the United States
6,De Land,successful,US,the United States
7,Indianapolis,live,US,the United States
8,Grand Rapids,successful,US,the United States
9,Miami,failed,US,the United States


In [17]:
# Inspect 'creator' column information
df.creator[0]

{'id': 1984360892,
 'name': 'Felipe Garcia',
 'is_registered': None,
 'chosen_currency': None,
 'is_superbacker': None,
 'avatar': {'thumb': 'https://ksr-ugc.imgix.net/assets/017/344/415/2854e64bf720927babc73e4d686186cb_original.JPG?ixlib=rb-2.1.0&w=40&h=40&fit=crop&v=1498991353&auto=format&frame=1&q=92&s=b0b7954efdeb54319b1e4faa9b703dcf',
  'small': 'https://ksr-ugc.imgix.net/assets/017/344/415/2854e64bf720927babc73e4d686186cb_original.JPG?ixlib=rb-2.1.0&w=160&h=160&fit=crop&v=1498991353&auto=format&frame=1&q=92&s=b20d7bb4f7e472f70e2ba23ca346a4d7',
  'medium': 'https://ksr-ugc.imgix.net/assets/017/344/415/2854e64bf720927babc73e4d686186cb_original.JPG?ixlib=rb-2.1.0&w=160&h=160&fit=crop&v=1498991353&auto=format&frame=1&q=92&s=b20d7bb4f7e472f70e2ba23ca346a4d7'},
 'urls': {'web': {'user': 'https://www.kickstarter.com/profile/1984360892'},
  'api': {'user': 'https://api.kickstarter.com/v1/users/1984360892?signature=1571370022.6e2fbc9ffaad29f35e678667bab789d0d6859abd'}}}

In [18]:
%%time
# Retract creator id and creator name into dataframe
creator_id_lst = [None]*len(df)
creator_name_lst = [None]*len(df)
for index, row in df.iterrows():
    if type(row['creator']) == dict:
        creator_id_lst[index] = row['creator']['id']
        creator_name_lst[index] = row['creator']['name']

Wall time: 21.3 s


In [19]:
df['creator_id'] = creator_id_lst
df['creator_name'] = creator_name_lst

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205227 entries, 0 to 205226
Data columns (total 44 columns):
backers_count               205227 non-null int64
blurb                       205227 non-null object
category                    205227 non-null object
converted_pledged_amount    205227 non-null int64
country                     205227 non-null object
country_displayable_name    205227 non-null object
created_at                  205227 non-null datetime64[ns]
creator                     205227 non-null object
currency                    205227 non-null object
currency_symbol             205227 non-null object
currency_trailing_code      205227 non-null bool
current_currency            205227 non-null object
deadline                    205227 non-null datetime64[ns]
disable_communication       205227 non-null bool
friends                     108 non-null object
fx_rate                     205227 non-null float64
goal                        205227 non-null float64
id           

In [21]:
# Inspect the 4 columns with only 108 non-null object
question = ['is_starred', 'friends', 'is_backing', 'permissions']
for i in question:
    print(df[i].value_counts())

False    105
True       3
Name: is_starred, dtype: int64
[]    108
Name: friends, dtype: int64
False    108
Name: is_backing, dtype: int64
[]    108
Name: permissions, dtype: int64


In [23]:
df_qdna = df.loc[:, question].dropna(axis=0, how='all')
df_qdna.shape

(108, 4)

In [24]:
df_qdna[df_qdna.is_starred==True]

,is_starred,friends,is_backing,permissions
54147,True,[],False,[]
115548,True,[],False,[]
115928,True,[],False,[]


With the above inspection, it seems for those four strange columns, only the same 108 rows have entries, as we use dropna with how = 'all' and still 108 remains, and from those 108 entries, only is_starred shows 3 True entries and 105 False entries, the other three columns only provide the same entries, all False for is_backing, and [] for 'friends' or 'permissions'. Given the number of entries from those columns is too small compared to the dataframe, and the obscure content within those columns, we decide to drop them from this analysis.

In [25]:
# Inspect distribution for other bool columns
bool = ['disable_communication', 'is_starrable', 'spotlight', 'staff_pick']
for i in bool:
    print(df[i].value_counts())

False    204598
True        629
Name: disable_communication, dtype: int64
False    198402
True       6825
Name: is_starrable, dtype: int64
True     116127
False     89100
Name: spotlight, dtype: int64
False    178981
True      26246
Name: staff_pick, dtype: int64


In [26]:
# Inspect text content
df.loc[:,['name', 'slug', 'blurb']].head()

,name,slug,blurb
0,Playable Certification (Canceled),playable-certification,Authenticate and certificate collectible gamin...
1,Deadiron's Next Album!,deadirons-next-album,The latest in Deadiron heavy metal! Help us fu...
2,Lindsey Holland Rick Holland Christmas Project...,lindsey-holland-rick-holland-christmas-project...,Lindsey Holland Debut Vocal Disc with the Litt...
3,The Belly of the Whale is the Belly of You,the-belly-of-the-whale-is-the-belly-of-you,Whales are washing up around the world with de...
4,"Quartet Style... Recording my jazz vocal CD ""B...",quartet-style-recording-my-jazz-vocal-cd-back-...,Spirited and talented crooner personality reac...


For text content, it seems slug is just name of project connected with hyphen, doesn't seem to be very useful, so we dropped this column to save space

In [27]:
# Select columns to drop as they don't seem to be necessary for this project, 
# but still save them in a seperate json file for future reference
drop_list = ['converted_pledged_amount', 'currency_trailing_code', 'current_currency', 'fx_rate', 
             'category', 'source_url', 'urls', 'slug', 'photo', 'profile', 'creator', 'location', 'usd_type'] + question
keep_col = [i for i in df.columns if i not in drop_list]

In [29]:
# Check information on kept columns
df[keep_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205227 entries, 0 to 205226
Data columns (total 27 columns):
backers_count               205227 non-null int64
blurb                       205227 non-null object
country                     205227 non-null object
country_displayable_name    205227 non-null object
created_at                  205227 non-null datetime64[ns]
currency                    205227 non-null object
currency_symbol             205227 non-null object
deadline                    205227 non-null datetime64[ns]
disable_communication       205227 non-null bool
goal                        205227 non-null float64
id                          205227 non-null int64
is_starrable                205227 non-null bool
launched_at                 205227 non-null datetime64[ns]
name                        205227 non-null object
pledged                     205227 non-null float64
spotlight                   205227 non-null bool
staff_pick                  205227 non-null bool
state 

In [30]:
df.to_json('data/df_cleaned_full.json')

In [31]:
df.to_csv('data/df_cleaned.csv', header=True, columns=keep_col)